In [ ]:
import re
import nltk
import joblib
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

##load data

In [ ]:
data=pd.read_csv("/content/netflix_titles.csv", encoding='latin1')

##EDA

In [ ]:
print(f"data shape : {data.shape}")
print(f"data of columns : {data.columns}")
print(f"info abute data : {data.info()}")
print(f"data describe : {data.describe()}")
print("first 5 rows in data")
data.head()

data shape : (8809, 26)
data of columns : Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description',
       'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unnamed: 25'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8809 entries, 0 to 8808
Data columns (total 26 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       8809 non-null   object 
 1   type          8809 non-null   object 
 2   title         8809 non-null   object 
 3   director      6175 non-null   object 
 4   cast          7984 non-null   object 
 5   country       7978 non-null   object 
 6   date_added    8799 non-null   object 
 7   release_year  8809 non-null   int64  
 8   ra

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data=data.drop(data[['Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15','Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19','Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23','Unnamed: 24', 'Unnamed: 25']],axis=1)

In [ ]:
print(f"data shape : {data.shape}")
print(f"data of columns : {data.columns}")
print(f"info abute data : {data.info()}")
print(f"data describe : {data.describe()}")
print("first 5 rows in data")
data.head()

data shape : (8809, 12)
data of columns : Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8809 entries, 0 to 8808
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8809 non-null   object
 1   type          8809 non-null   object
 2   title         8809 non-null   object
 3   director      6175 non-null   object
 4   cast          7984 non-null   object
 5   country       7978 non-null   object
 6   date_added    8799 non-null   object
 7   release_year  8809 non-null   int64 
 8   rating        8805 non-null   object
 9   duration      8806 non-null   object
 10  listed_in     8809 non-null   object
 11  description   8809 non-null   object
dtypes: int64(1), object(11)
memory usage: 826.0+ KB
info abute data : None
data descri

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [ ]:
for col in data.select_dtypes(include=['object']).columns:
  print("columns name : ",col)
  values_in_col=data[col].value_counts()
  print(values_in_col)
  print("_________________________________________________")

columns name :  show_id
show_id
s8809    1
s1       1
s2       1
s3       1
s4       1
        ..
s14      1
s13      1
s12      1
s11      1
s10      1
Name: count, Length: 8809, dtype: int64
_________________________________________________
columns name :  type
type
Movie      6132
TV Show    2677
Name: count, dtype: int64
_________________________________________________
columns name :  title
title
22-Jul                                                           2
Feb-09                                                           2
15-Aug                                                           2
Trailer Park Boys: Drunk, High and Unemployed: Live in Austin    1
The Ridiculous 6                                                 1
                                                                ..
Azali                                                            1
Dragons: Rescue Riders                                           1
Fifty: The Series                                          

##data clineing


In [ ]:
#take copy from oragnal data
df=data.copy()

In [ ]:
df.isnull().sum()

,0
show_id,0
type,0
title,0
director,2634
cast,825
country,831
date_added,10
release_year,0
rating,4
duration,3


In [ ]:
#handling missing values
df['director']=df['director'].fillna(value=df['director'].mode().iloc[0])
df['country']=df['country'].fillna(value=df['country'].mode().iloc[0])
df['date_added']=df['date_added'].fillna(value=df['date_added'].mode().iloc[0])
df['rating']=df['rating'].fillna(value=df['rating'].mode().iloc[0])
df['duration']=df['duration'].fillna(value=df['duration'].mode().iloc[0])
df=df.dropna()


In [ ]:
print(df.shape)
df.isnull().sum()


(7984, 12)


,0
show_id,0
type,0
title,0
director,0
cast,0
country,0
date_added,0
release_year,0
rating,0
duration,0


In [ ]:
for col in df:
  values_in_col1=df[col].unique()
  print(f"columns name : {col} \n {values_in_col1} \n _________________________________________")

columns name : show_id 
 ['s2' 's3' 's5' ... 's8807' 's8808' 's8809'] 
 _________________________________________
columns name : type 
 ['TV Show' 'Movie'] 
 _________________________________________
columns name : title 
 ['Blood & Water' 'Ganglands' 'Kota Factory' ... 'Zubaan'
 'Parasyte: The Grey' 'Serena'] 
 _________________________________________
columns name : director 
 ['Rajiv Chilaka' 'Julien Leclercq' 'Mike Flanagan' ... 'Peter Hewitt'
 'Mozez Singh' 'Yeon Sang-ho'] 
 _________________________________________
columns name : cast 
 ['Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Greeff, Xolile Tshabalala, Getmore Sithole, Cindy Mahlangu, Ryle De Morny, Greteli Fincham, Sello Maake Ka-Ncube, Odwa Gwanya, Mekaila Mathys, Sandi Schultz, Duane Williams, Shamilla Miller, Patrick Mofokeng'
 'Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabiha Akkari, Sofia Lesaffre, Salim Kechiouche, Noureddine Farihi, Geert Van Rampelberg, Bakary Dio

In [ ]:
df.duplicated().sum()

np.int64(0)

In [ ]:
#featuer extract
df['movive_info']=df['type'] + " " + df['title'] + " " + df['listed_in'] + " " + df['duration'] + " "  + df['director'] + " " + df['description'] + " " + df['country'] + " " + df['cast']


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
port_stem=PorterStemmer()
def stemming (movive_info):
  text_stem=re.sub('[^a-zA-Z]',' ',movive_info)
  text_stem=text_stem.lower().strip().split()
  text_stem=[port_stem.stem(word) for word in text_stem if not word in stopwords.words('english') ]
  text_stem=' '.join(text_stem)
  return text_stem

In [ ]:
df['movive_info']=df['movive_info'].apply(stemming)


In [ ]:
vectorizer=TfidfVectorizer()
content=df['movive_info'].values
tfidf_matrix=vectorizer.fit_transform(content)

In [ ]:
joblib.dump(vectorizer,"tfidf_vectorizer.pkl")
joblib.dump(tfidf_matrix,"tfidf_matrix.pkl")
df.to_csv("preprocessing_data.csv",index=False)